# Homework 3 - Application Building

- Python Computing for Data Science (2022)

- Due Tuesday Feb 15 (8pm)

### Casey Lam, casey_lam@berkeley.edu

### Collaborators: #hw3-discussion channel

## CalCalc

Write a module called `CalCalc`, with a method called `calculate` that evaluates any string passed to it, and can be used from either the command line (using `argparse` with reasonable flags) or imported within Python. Feel free to use something like `eval()`, but be aware of some of the nasty things it can do, and make sure it doesn’t have too much power:  http://nedbatchelder.com/blog/201206/eval_really_is_dangerous.html. Perhaps explore the use of `numexpr` to constrain the landscape of possible uses to math expressions.

EXAMPLE:
```bash
$ python CalCalc.py -s '34*28'
$ 952
```
 AND, from within Python
 
```python
>>> from CalCalc import calculate
>>> calculate('34*20')
>>> 952
```

In [34]:
%%writefile calcalc/CalCalc.py

import argparse
import numexpr
import urllib
import requests
import pdb
import re

number_dict = {'one' : '*1.',
               'ten' : '*10.',
               'hundred' : '*100.',
               'thousand' : '*1000.',
               'million' : '*10.**6',
               'billion' : '*10.**9',
               'trillion' : '*10.**12',
               'quadrillion' : '*10.**15',
               'quintillion' : '*10.**18',
               'sextillion' : '*10.**21',
               'septillion' : '*10.**24'}

def calculate(in_str, return_float=False):
    # Evaluate remotely using wolfram.
    if return_float:
        # Convert question to URL and sent to wolfram
        url_str = urllib.parse.quote_plus(in_str)
        app_id = 'Q9RQK4-QK54QKTJ72'
        url_wolfram = 'https://api.wolframalpha.com/v2/query?input=' + url_str \
                        + '&appid=' + app_id + '&output=json&scanner=Data,Identity'
        try:
            answer = requests.get(url_wolfram)

            # Get the answer string, location depends on whether the scanner is Identity or Data
            if answer.json()['queryresult']['pods'][0]['scanner'] == 'Identity':
                answer_text = answer.json()['queryresult']['pods'][1]['subpods'][0]['plaintext']
            elif answer.json()['queryresult']['pods'][0]['scanner'] == 'Data':
                answer_text = answer.json()['queryresult']['pods'][0]['subpods'][0]['img']['title']
            print('Answer (direct from Wolfram): ', answer_text)

            # Simplify answers that are convoluted
            answer_text = answer_text.split('\n', 1)[0] # skip parenthetical clarifications, units.
            answer_text = answer_text.split(' (', 1)[0] # skip blah blah details. THE SPACE IS IMPORTANT
            answer_text = answer_text.split('to', 1)[0] # give lower range

            # Convert math symbols to be python.
            answer_text = answer_text.replace('×', '*')
            answer_text = answer_text.replace('^', '**')
            #answer_text = answer_text.replace(' to the ', '**')
            #answer_text = answer_text.replace(' times ', '*')
            
            # Convert numbers spelled out in words to numbers
            if any([x in answer_text for x in number_dict.keys()]):
                for key in number_dict.keys():
                    if key in answer_text:
                        answer_text = answer_text.replace(key, number_dict[key])
            
            # Fix potential overflow due to integers only.
            if '**' in answer_text:
                if '.' not in answer_text:
                    answer_text += '.'
            
            # Clean up anything that is not a number or relevant math symbol
            answer_text = re.sub('[^1234567890*.]', '', answer_text)
            answer = float(numexpr.evaluate(answer_text))
            print('Answer (float): ', answer)
            return answer
        
        except:
            print('This question\'s answer isn\'t convertable to a numerical string. \n' + 
                  'Try rephrasing your question (e.g. specify units of the result).')

    # Evaluate locally using python.
    else:
        try:
            # Fix potential overflow due to integers only.
            if '**' in in_str:
                if '.' not in in_str:
                    in_str += '.'
            answer = float(numexpr.evaluate(in_str))
            print(answer)
            return answer
        except:
            print('This expression can\'t be evaluated numerically. \n' + 
                  'Did you mean to evaluate with wolfram? \n' +
                  'If not, check your question for typos.')

def test0():
    assert abs(4. - calculate('2**2')) < 0.001

def test1():
    assert abs(206. - calculate('how many bones in the human body', return_float=True)) < 0.001
    
def test2():
    assert abs(12. - calculate('convert 1 feet to inches', return_float=True)) < 0.001
    
def test3():
    assert abs(2.e204 - calculate('10e3*2e200')) < 10.e203
    
def test4():
    assert abs(7.3459e22 - calculate('mass of the moon in kg', return_float=True)) < 10.e21
    
def test5():
    assert abs(100. - calculate('water boiling point in celsius', return_float=True)) < 1
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Write something useful here.')
    parser.add_argument('-s', action='store', dest='question_python', 
                        help='Numbers', default=None)
    parser.add_argument('-w', action='store', dest='question_wolfram',
                        help='Words', default=None)

    results = parser.parse_args()
    
    
    if (results.question_python != None) & (results.question_wolfram != None):
        raise Exception('Make up your mind! You can only set one flag.')

    if results.question_wolfram is None:
        return_float=False
        question = results.question_python
    else:
        return_float=True     
        question = results.question_wolfram
        
    calculate(question, return_float=return_float)

Overwriting calcalc/CalCalc.py


In [35]:
!python calcalc/CalCalc.py -w 'CHICKENS!!!!!'

This question's answer isn't convertable to a numerical string. 
Try rephrasing your question (e.g. specify units of the result).


In [36]:
# All sorts of things I tried.
!python calcalc/CalCalc.py -w 'how many days in a year'
!python calcalc/CalCalc.py -w 'how many inches in 1 foot'
!python calcalc/CalCalc.py -w 'how many miles in 2.6 km'
!python calcalc/CalCalc.py -w 'how many bones in the body'
!python calcalc/CalCalc.py -w 'wavelength of red light'
!python calcalc/CalCalc.py -w 'wavelength of red light IN FEET'
!python calcalc/CalCalc.py -w 'mass of the moon in kg'
!python calcalc/CalCalc.py -w 'how many pandas'
!python calcalc/CalCalc.py -w 'how many countries'
!python calcalc/CalCalc.py -w 'price of gold'
!python calcalc/CalCalc.py -w 'price of gold in dollars per oz'
!python calcalc/CalCalc.py -w 'CHICKENS!!!!!'
!python calcalc/CalCalc.py -w 'how many people on earth' 
!python calcalc/CalCalc.py -w 'how many grains of sand' 
!python calcalc/CalCalc.py -s 'mass of the moon in kg'
!python calcalc/CalCalc.py -s '1*10**100'
!python calcalc/CalCalc.py -s '1.*10**100'

Answer (direct from Wolfram):  365 days
Answer (float):  365.0
Answer (direct from Wolfram):  12 inches
Answer (float):  12.0
Answer (direct from Wolfram):  1.62 miles
Answer (float):  1.62
Answer (direct from Wolfram):  206
Answer (float):  206.0
Answer (direct from Wolfram):  (620 to 750) nm (nanometers)
Answer (float):  620.0
Answer (direct from Wolfram):  (2.03×10^-6 to 2.46×10^-6) feet
Answer (float):  2029999.9999999998
Answer (direct from Wolfram):  7.3459×10^22 kg (kilograms)
Answer (float):  7.3459e+22
Answer (direct from Wolfram):  1800
Answer (float):  1800.0
Answer (direct from Wolfram):  206
(according to Article 1 of the Montevideo Convention of 1933, in which a state must have: (1) a permanent population, (2) a defined territory, (3) a government, and (4) a capacity to enter into relations with the other states)
Answer (float):  206.0
Answer (direct from Wolfram):  $1860/oz t (US dollars per troy ounce) (Sunday, February 13, 2022)
Answer (float):  1860.0
Answer (direct f

In [37]:
# pip install -U pytest
!pytest ./calcalc/CalCalc.py

============================= test session starts ==============================
platform linux -- Python 3.9.9, pytest-7.0.1, pluggy-1.0.0
rootdir: /home/jovyan/python-ay250-homeworks/hw_3
plugins: anyio-3.4.0
collected 6 items                                                              

calcalc/CalCalc.py ......                                                [100%]

============================== 6 passed in 4.29s ===============================


### Add Wolfram

To make this more awesome, have your function interact with the Wolfram|Alpha API to ask it what it thinks of the difficult questions.  To make this work, experiment with `urllib2` and a URL like this:
'http://api.wolframalpha.com/v2/query?input=XXXXX&appid=UAGAWR-3X6Y8W777Q'
where you replace the XXXXX with what you want to know.  NOTE: the ‘&appid=UAGAWR-3X6Y8W777Q’ part is vital; it is a W|A AppID I got for the class.  Feel free to use that one, or you can get your own and read more about the API, here:   http://products.wolframalpha.com/api/
And you can explore how it works here:  http://products.wolframalpha.com/api/explorer.html

EXAMPLE:

```bash
$ python CalCalc.py -w 'mass of the moon in kg'
7.3459e+22
```

AND, from within Python

```python
>>> from CalCalc import calculate
>>> calculate('mass of the moon in kg',  return_float=True) * 10
>>> 7.3459e+23
```

## Adding it to Github

Start a github project for CalCalc. Include a setup.py, README.txt, LICENSE.txt, MANIFEST.in, etc. and turn your module into a proper Python Distribution, so that we can install it and use it. See https://packaging.python.org/en/latest/guides/distributing-packages-using-setuptools/ 

Example Folder Hierarchy:
```bash
Your_Homework3_Folder/calcalc
                      |--> CalCalc.py
                      |--> __init__.py
Your_Homework3_Folder/setup.py
Your_Homework3_Folder/README.txt
...
```
Include at least 5 test functions in CalCalc.py, and test with `pytest`, to make sure it behaves the way you think it should.

EXAMPLE `CalCalc.py`:
```python
# ...
def calculate([...]):
    [...]

def test_1():
    assert abs(4. - calculate('2**2')) < 0.001
```

When grading, we will create a virtual environment and attempt to install your module by running:

```bash
pip install build
```

**Answer:** Link to package at https://pypi.org/project/calcalc/

### CalCalc on CI

Get your project working with GitHub Actions and make sure your tests are run and pass. Give us a link to you GH actions for your site here (e.g. https://github.com/profjsb/PyAdder/actions):

### **(Bonus/Extra Credit)** 

  Get your project working on Azure, AWS or Google Compute Cloud with a Flask front-end. You can use the example from class as a template. Start a VM on one of these PaaS. A user should be able to submit their calcalc query on a form (hosted on your VM) and get the result back.

You should be able to add an `app.py` (with Flask) into your CalCalc project. Be sure to open up the port on the VM that you are serving on. Let us know the URL to your app here: